In [1]:
import logging
import os
import gzip

import pandas
import easydict

import analysis

In [2]:
analysis.logger.propagate = False

In [3]:
commit = '153688869fb87412dd716ac27994e51baeb16078'
url = 'https://raw.githubusercontent.com/dhimmel/stargeo/{}/data/queries.tsv'.format(commit)
query_df = pandas.read_table(url)
query_df.head()

,slim_id,slim_name,case_query,control_query
0,DOID:0050156,idiopathic pulmonary fibrosis,IPF == 'IPF',IPF_control == 'IPF_control'
1,DOID:0050741,alcohol dependence,alcoholism == 'alcoholism',alcoholism_control == 'alcoholism_control'
2,DOID:0050742,nicotine dependence,Smoker == 'Smoker',Nonsmoker == 'Nonsmoker'
3,DOID:1024,leprosy,borderline_leprosy == 'borderline_leprosy' or ...,leprosy_control == 'leprosy_control'
4,DOID:10283,prostate cancer,PC == 'PC' or PC_tissue_case == 'PC_tissue_case',PC_control == 'PC_control' or PC_tissue_contro...


In [4]:
for i, row in query_df.iterrows():
    print i, row.slim_id, row.slim_name

    name = row.slim_id.replace(':', '_')
    params = easydict.EasyDict(
        analysis_name = name,
        case_query = row.case_query,
        control_query = row.control_query,
        modifier_query = '',
        min_samples = 3,
    )
    
    directory = 'data/{}'.format(params.analysis_name)
    if not os.path.isdir(directory):
        os.mkdir(directory)

    logfile = logging.FileHandler('{}/log.txt'.format(directory))
    logfile.setLevel(logging.DEBUG)
    analysis.logger.addHandler(logfile)
    try:
        sample_df, result_df = analysis.perform_analysis(params)
    except:
        sample_df, result_df = None, None
    analysis.logger.removeHandler(logfile)
    if sample_df is not None:
        path = '{}/samples.tsv'.format(directory)
        sample_df.to_csv(path, index=False, sep='\t')
        
    if result_df is not None:
        path = '{}/meta.tsv'.format(directory)
        result_df.to_csv(path, index=False, sep='\t', float_format='%.5g')

analysis.py:164: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  df = df.convert_objects(convert_numeric=True)
/home/dhimmels/anaconda3/envs/python2/lib/python2.7/site-packages/pandas/core/indexing.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/dhimmels/anaconda3/envs/python2/lib/python2.7/site-packages/pandas/core/index.py:4281: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return np.sum(name == np.asarray(self.names)) > 1
/home/dhimmels/anaconda3/envs/python2/lib/python2.7/site-packages/funcy/debug.py:100: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_me

0 DOID:0050156 idiopathic pulmonary fibrosis
1 DOID:0050741 alcohol dependence
2 DOID:0050742 nicotine dependence
3 DOID:1024 leprosy
4 DOID:10283 prostate cancer
5 DOID:10534 stomach cancer
6 DOID:10608 celiac disease
7 DOID:10652 Alzheimer's disease
8 DOID:10763 hypertension
9 DOID:10871 age related macular degeneration
10 DOID:1094 attention deficit hyperactivity disorder
11 DOID:11054 urinary bladder cancer
12 DOID:11612 polycystic ovary syndrome
13 DOID:11714 gestational diabetes
14 DOID:11949 Creutzfeldt-Jakob disease
15 DOID:12306 vitiligo
16 DOID:12365 malaria
17 DOID:12849 autistic disorder
18 DOID:13223 uterine fibroid
19 DOID:1324 lung cancer
20 DOID:13241 Behcet's disease
21 DOID:13378 Kawasaki disease
22 DOID:14221 metabolic syndrome X
23 DOID:14227 azoospermia
24 DOID:14330 Parkinson's disease
25 DOID:1595 endogenous depression
26 DOID:1612 breast cancer
27 DOID:1686 glaucoma
28 DOID:1793 pancreatic cancer
29 DOID:1909 melanoma
30 DOID:1936 atherosclerosis
31 DOID:219 col

/home/dhimmels/anaconda3/envs/python2/lib/python2.7/site-packages/pandas/core/groupby.py:2649: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 0 but corresponding boolean dimension is 1
  labels = list(map(rep, self.grouper.recons_labels)) + [lab[inc]]


In [5]:
#analysis.get_analysis_df(params.case_query, params.control_query, params.modifier_query)